<a href="https://colab.research.google.com/github/JinFree/Thermo-and-Fluid-Engineering-Lab.1/blob/master/Thermo_and_Fluid_Engineering_Lab_1_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matplotlib

## Matplotlib Import

In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt

* Style Setting

In [0]:
plt.style.use('classic')

# 과제
## 4번
* 파일 - 새 Python3 노트를 통해 새 노트북을 만드시오.
* Python을 활용하여 구구단을 출력하는 함수를 만드시오.
 * 입력은 하나의 정수를 받으며, 출력은 해당 정수의 구구단을 총 9줄로 출력함.
* 파일 - Github에 사본 저장을 통해 1번 과제에서 복사한 저장소에 아래와 같은 이름으로 저장하시오.
 * 열유체공학실험-2016201278-HW04
 * 다른 제목인 경우 미제출 처리 될 수 있음.